In [9]:
#based on https://www.kaggle.com/keegil/keras-u-net-starter-lb-0-277
#https://www.kaggle.com/c/ultrasound-nerve-segmentation/overview


import os
import numpy as np

from skimage.io import imsave, imread
from skimage import transform

def load_data(datadir):
    train_data_path = os.path.join("", datadir)
    images = os.listdir(train_data_path)
    

    imgs = []
    imgs_mask = []

 
    for image_name in images:
        if 'mask' in image_name:
            continue
        image_mask_name = image_name.split('.')[0] + '_mask.tif'
        img = imread(os.path.join(train_data_path, image_name))
        img_mask = imread(os.path.join(train_data_path, image_mask_name))

        img = np.array([img])
        img_mask = np.array([img_mask])
        

        imgs.append(img)
        imgs_mask.append(img_mask)
        
        
        
        
    imgs_mask=np.array(imgs_mask)
    imgs=np.array(imgs)
        
    
    return imgs,imgs_mask

In [7]:
images,masks=load_data('train')
test=load_data('test')

/opt/anaconda3/lib/python3.7/site-packages/skimage/external/tifffile/tifffile.py:2618: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)


In [25]:
images=np.array(images)
masks=np.array(masks)
test=np.array(test)

In [18]:
images_train=[transform.resize(img,(64,64,1)) for img in images]
masks_train=[transform.resize(img,(64,64,1)) for img in masks]

In [50]:
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K

import tensorflow as tf

In [52]:
inputs = Input(images[0].shape)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
conv1=BatchNormalization()(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
conv2=BatchNormalization()(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
conv3=BatchNormalization()(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
conv4=BatchNormalization()(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

model = Model(inputs=[inputs], outputs=[conv10])

In [53]:
model.compile(optimizer='adam', loss='binary_crossentropy')
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_12 (InputLayer)            (None, 64, 64, 1)     0                                            
____________________________________________________________________________________________________
conv2d_174 (Conv2D)              (None, 64, 64, 32)    320         input_12[0][0]                   
____________________________________________________________________________________________________
conv2d_175 (Conv2D)              (None, 64, 64, 32)    9248        conv2d_174[0][0]                 
____________________________________________________________________________________________________
batch_normalization_2 (BatchNorm (None, 64, 64, 32)    128         conv2d_175[0][0]                 
___________________________________________________________________________________________

In [ ]:
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('model-dsbowl2018-1.h5', verbose=1, save_best_only=True)
results = model.fit(images, masks, validation_split=0.1, batch_size=16, epochs=50, 
                    callbacks=[earlystopper, checkpointer])

Train on 5071 samples, validate on 564 samples
Epoch 1/50
1568/5071 [========>.....................] - ETA: 586s - loss: 0.0702

In [ ]:
predicted_masks=model.predict(test)

In [ ]:
plt.imshow(predicted_masks[0])